<a href="https://colab.research.google.com/github/SriSatyaLokesh/some-app-for-whatsapp/blob/master/notebooks/analytics/Visualization_satya.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
uploaded = files.upload()

Saving WhatsApp Chat with CSE 2K16-2K20.txt to WhatsApp Chat with CSE 2K16-2K20.txt


# DataFrame

In [63]:
import pandas as pd
from itertools import chain, repeat
import re
from string import digits

chainer = chain.from_iterable
class Cleaner:
  def __init__(self,filename):
    self.filename = filename

  def _clean_data(self):
    chat_file = self.filename
    with open(chat_file) as file:
      all_msgs = file.readlines()
      raw_texts = []
      for line in all_msgs:
        if re.match("^\d{2}/\d{2}/\d{2,4},\s\d{1,2}:\d{2}\s[ap]*[m]*",line):
          date_split = line.split(' - ') #date and user
          user_split = date_split[1].split(':')
          if len(user_split) >= 2:
            raw_texts.append(line)
        else:
          try:
            raw_texts[-1] += line
          except Exception as err:
            # print(line,raw_texts)
            # print(err)
            pass

    return raw_texts 
      
  def _get_data(self):
    raw_texts = self._clean_data()
    raw_messages = dict()
    for line in raw_texts:
      date_split = line.split(' - ')
      user_split = date_split[1].split(':')
      user = user_split[0]
      if user not in raw_messages:
        raw_messages[user] = [line]
      else:
        raw_messages[user].append(line)

    df = pd.DataFrame({'user': list(chainer(repeat(k, len(v)) for k,v in raw_messages.items())),
                   'raw_text': list(chainer(raw_messages.values()))}) 
    return df
  def _preprocess_data(self,df,min_msgs = 10):
    df["raw_message"] = df.apply(lambda df : "".join(df['raw_text'].split(":")[2:]),axis=1)
    df["raw_message"] = df.apply(lambda df : df['raw_message'][:-1],axis=1)
    remove_digits = str.maketrans('', '', digits)
    df["text_only_message"] = df.apply(lambda df : df["raw_message"].encode('ascii', 'ignore').decode('ascii'),axis=1)
    df["text_only_message"] = df.apply(lambda df : df["text_only_message"].translate(remove_digits),axis=1)
    df.loc[(df['text_only_message'] == " This message was deleted") | (df['text_only_message'] == " <Media omitted>"),'text_only_message'] = ""
    df = df.groupby('user').filter(lambda x : len(x) > min_msgs)

    df['datetime'] =  df.apply(lambda df : df['raw_text'].split(" - ")[0],axis=1)

    temp = ["%d/%m/%Y, %I:%M %p" , "%d/%m/%y, %I:%M %p" , "%d/%m/%Y, %H:%M" , "%d/%m/%y, %H:%M" ,
            "%d/%Y/%m, %I:%M %p" , "%d/%y/%m, %I:%M %p" , "%d/%Y/%m, %H:%M" , "%d/%y/%m, %H:%M" ,
            "%Y/%m/%d, %I:%M %p" , "%y/%m/%d, %I:%M %p" , "%Y/%m/%d, %H:%M" , "%y/%m/%d, %H:%M" ,
            "%Y/%d/%m, %I:%M %p" , "%y/%d/%m, %I:%M %p" , "%Y/%d/%m, %H:%M" , "%y/%d/%m, %H:%M" ,
            "%m/%Y/%d, %I:%M %p" , "%m/%y/%d, %I:%M %p" , "%m/%Y/%d, %H:%M" , "%m/%y/%d, %H:%M" ,
            "%m/%d/%Y, %I:%M %p" , "%m/%d/%y, %I:%M %p" , "%m/%d/%Y, %H:%M" , "%m/%d/%y, %H:%M"]

    for formats in temp:       
      try:
        df['datetime'] = pd.to_datetime(df['datetime'], format=formats)
      except:
        continue
    df['date'] = df['datetime'].dt.date
    df['hour'] = df['datetime'].dt.hour
    df['weekday'] = df['datetime'].dt.weekday
    return df

chat_file = "WhatsApp Chat with CSE 2K16-2K20.txt"
cleaner = Cleaner(chat_file)
df = cleaner._get_data()
df = cleaner._preprocess_data(df)
df

,user,raw_text,raw_message,text_only_message,datetime,date,hour,weekday
0,+91 81432 75536,"12/10/2019, 11:21 am - +91 81432 75536: Remain...",Remainder,Remainder,2019-10-12 11:21:00,2019-10-12,11,5
1,+91 81432 75536,"25/10/2019, 3:30 pm - +91 81432 75536: <Media ...",<Media omitted>,,2019-10-25 15:30:00,2019-10-25,15,4
2,+91 81432 75536,"25/10/2019, 3:30 pm - +91 81432 75536: <Media ...",<Media omitted>,,2019-10-25 15:30:00,2019-10-25,15,4
3,+91 81432 75536,"25/10/2019, 3:33 pm - +91 81432 75536: Dear st...",Dear students these are shortlisted candidate...,Dear students these are shortlisted candidate...,2019-10-25 15:33:00,2019-10-25,15,4
4,+91 81432 75536,"25/10/2019, 3:38 pm - +91 81432 75536: The sho...",The shortlisted students for Hitachi consulti...,The shortlisted students for Hitachi consulti...,2019-10-25 15:38:00,2019-10-25,15,4
...,...,...,...,...,...,...,...,...
318,+91 99126 48686,"15/04/2020, 10:19 am - +91 99126 48686: Guys l...",Guys last 2 minutes,Guys last minutes,2020-04-15 10:19:00,2020-04-15,10,2
319,+91 99126 48686,"15/04/2020, 10:19 am - +91 99126 48686: Exam w...",Exam will closes,Exam will closes,2020-04-15 10:19:00,2020-04-15,10,2
320,+91 99126 48686,"15/04/2020, 10:30 am - +91 99126 48686: <Media...",<Media omitted>,,2020-04-15 10:30:00,2020-04-15,10,2
321,+91 99126 48686,"15/04/2020, 10:38 am - +91 99126 48686: <Media...",<Media omitted>,,2020-04-15 10:38:00,2020-04-15,10,2


In [0]:
import plotly.express as px

# Each user message count in week plots

In [15]:
import plotly.graph_objects as go

weekdays = ['Monday','Tuesday','Webnesday','Thrusday','Friday','Saturday','Sunday']

user_plots = go.Figure()

for user in df['user'].unique()[:3]:
  user_plots.add_trace(go.Scatterpolar(
          r = df[df['user'] == user].groupby('weekday').size(),
          theta = weekdays,
          fill = 'toself',
          name = user
    ))
  
  user_plots.update_layout(
      polar=dict(
        radialaxis=dict(
          visible=False,
          range=[0,40],
        )),
      showlegend=True,
      template="plotly_dark"
    )

user_plots.show()

In [29]:
msgs_count = pd.DataFrame({'msg_count' : df.groupby(df['date'].dt.date).size()})
msgs_count = msgs_count.reset_index(drop = False)
msgs_count

,date,msg_count
0,2019-10-12,1
1,2019-10-25,5
2,2019-10-31,3
3,2019-11-01,2
4,2019-11-12,1
...,...,...
78,2020-04-12,3
79,2020-04-13,4
80,2020-04-15,11
81,2020-04-23,7


In [31]:
msgs_count_fig = go.Figure()

msgs_count_fig.add_trace(go.Scatter(
        x = msgs_count['date'],
        y = msgs_count['msg_count'],
        mode='lines+markers'
  ))
  
msgs_count_fig.update_layout(
      showlegend=True,
      template="plotly_dark"
    )

msgs_count_fig.show()

In [72]:
def get_top_n_users(df,n=3):
  df = pd.DataFrame({'count' : df.groupby(["user"]).size()})
  df = df.nlargest(n,columns=["count"])
  df = df.reset_index(drop = False)
  return df

0      2019-10-12
1      2019-10-25
2      2019-10-25
3      2019-10-25
4      2019-10-25
          ...    
318    2020-04-15
319    2020-04-15
320    2020-04-15
321    2020-04-15
322    2020-04-15
Name: date, Length: 229, dtype: object

In [95]:
users = get_top_n_users(df,3)
print(users)
msg_count_per_user  = go.Figure()

for user in users['user']:
  msg_count_per_user.add_trace(go.Scatter(
          y = df[df['user'] == user].groupby(df['date']).size(),
          x = df['date'],
          mode='lines+markers',
          name = user,
    ))
  

msg_count_per_user.update_layout(
      showlegend=True,
      template="plotly_dark"
    )

msg_count_per_user.show()

              user  count
0  +91 81432 75536    138
1  +91 99126 48686     31
2  +91 99789 67155     23
